# Create New Datastreams of Ranchbot Devices

This script was created following the integration of Ranchbot data to Dendra. Currently a webhook pipes Ranchbot data as it arrives into Dendra tables. To set up datastreams for existing Ranchbot devices as well as to prepare for anticipated new installations, this script creates new datastreams on Dendra to surface the stored Ranchbot data. Each well at the Dangermond Preserve already has a station on Dendra.

When new ranchbot devices are installed and datastreams need to be created for them on Dendra follow this workflow:

1. Reach out to Scott @ Dendra to let him know that data should be arriving on Dendra from the new ranchbots.
2. Scott will send you a spreadsheet with information about where that data is stored on Dendra. This table will have fields for sensor_depth for you to fill out.
3. Either run this script on a csv generated from the filled out spreadsheet, or ask Scott to set up the datastreams. 

# Load Libraries

In [48]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy
import sys
import getpass
import json
import requests

den_utils_path = "../ArcGIS Tools/Utils/"
sys.path.append(den_utils_path)
import well_data_dendra_helpers as dendra

url = 'https://api.dendra.science/v2/'  
headers = {"Content-Type":"application/json"}

In [49]:
#Authenticate Dendra User:
# If you have a login and the data is not public, you must authenticate using your Dendra login
dendra_user = "YOURDENDRAEMAIL"
dendra_pass = getpass.getpass()


creds = {
    'email': dendra_user,
    'strategy': 'local',
    'password': dendra_pass
}

r = requests.post(url+'authentication', json=creds)
if r.status_code != 201:
    raise AssertionError("Dendra authentication failed, check your email/password")

token = r.json()['accessToken']
headers['Authorization'] = token

In [71]:
#Point to folder where jsons for dendra API can be stored
if not os.path.exists("./DendraJSONs"):
    os.mkdir("./DendraJSONs")
    os.mkdir("./DendraJSONs/Stations")
    os.mkdir("./DendraJSONs/Datastreams")
    os.mkdir("./DendraJSONs/Annotations")

if not os.path.exists("./DendraJSONs/Requests"):
    os.mkdir("./DendraJSONs/Requests")
    
dendra_dir = "./DendraJSONs/"



# Fetch metadata on datastreams to create

In [88]:
datastreams = pd.read_csv("TNCRanchbotDendraTables.csv")
column_to_template = {
    "sensor_109_mm": "ranchbot_dailyrainfall.json",
    "sensor_127_m": "ranchbot_depthtogroundwater.json",
    "sensor_107_degc":  "ranchbot_watertemperature.json"
}

In [83]:
#Iterate through datastreams to create and create new jsons
for _i, data in datastreams.iterrows():
    datastream_template_file = column_to_template.get(data['column'][:-7])
    if not datastream_template_file:
        continue

    with open(f"{dendra_dir}/Templates/{datastream_template_file}") as datastream_template_raw:
        datastream_template = deepcopy(json.load(datastream_template_raw))
    
    station_metadata = requests.get(url + "stations/", headers=headers, params = {"_id":data['station_id']}).json()['data'][0]

    #Populate datastream metadata
    datapoints_query = datastream_template['datapoints_config'][0]['params']['query']
    datapoints_query['fc'] = data['fc']
    datapoints_query['sc'] = datapoints_query['sc'].replace("VARIABLEID", data['column'].split("_")[-1])
    datapoints_query['source'] = datapoints_query['source'].replace("DEVICEID", data['fc'])
    datastream_template['datapoints_config'][0]['params']['query'] = datapoints_query

    datastream_template['description'] = datastream_template['description'].replace("DEVICEID", data['fc'].split("_")[1])
    datastream_template['description'] = datastream_template['description'].replace("STATIONNAME", station_metadata['name'])
    datastream_template['station_id'] = station_metadata['_id']

    #If depth to groundwater, set sensor depth
    if "sensor_127_m" in data['column']:
        datastream_template['attributes']['ranchbot_sensor_depth'] = data['sensor_depth']

    datastream_template["geo"] = station_metadata['geo']
    
    # Write the JSON objects to a file
    with open(f"{dendra_dir}/Requests/{station_metadata['slug']}-{data['column']}.datastream.json", 'w') as json_file:
        json.dump(datastream_template, json_file, indent=4)

In [84]:
#Run this cell to push those stations to Dendra!
!den meta push-datastreams --filespec=$dendra_dir/Requests/*.datastream.json --verbose


Created:       ./DendraJSONs/Requests/dangermond-escondido-3-sensor_107_degc_214573.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-escondido-3-sensor_109_mm_225791.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-escondido-3-sensor_127_m_225790.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-escondido5-sensor_107_degc_195322.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-escondido5-sensor_109_mm_197851.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-escondido5-sensor_127_m_196054.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-lowerjalamavaqueros-sensor_107_degc_214589.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-lowerjalamavaqueros-sensor_109_mm_221165.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-lowerjalamavaqueros-sensor_127_m_221164.datastream.json
Created:       ./DendraJSONs/Requests/dangermond-oaks2-sensor_107_degc_214653.datastream.json